In [138]:
import requests
import pandas as pd
headers = {'api_key': '5d7494bbde0c411981d3832703734b09'}

In [139]:
# explore all capital bike api
# https://gbfs.capitalbikeshare.com/gbfs/gbfs.json
# official explaination: https://github.com/NABSA/gbfs/blob/master/gbfs.md#system_regionsjson
#s = requests.get('https://gbfs.capitalbikeshare.com/gbfs/gbfs.json')
#ss = s.json()
#for i in ss['data']:
    #print()
    #for j in ss['data'][str(i)]['feeds']:
        #print(j)

In [124]:
def get_bus_location():
    '''
    get all capital bike station location information in DC
    '''
    system_regions = requests.get('https://gbfs.capitalbikeshare.com/gbfs/en/system_regions.json')
    station_information = requests.get('https://gbfs.capitalbikeshare.com/gbfs/en/station_information.json')
    #region_dic
    region_dic = {}
    for i in system_regions.json()['data']['regions']:
        region_dic[i['region_id']] = i['name']
    #dataframe   
    tmp = []
    for item in station_information.json()['data']['stations']:
        region_id = item['region_id']
        region_idx.append(region_id)

        if str(region_id) in region_dic.keys():
            region = region_dic[str(region_id)]
        tmp.append([item['station_id'],item['name'],item['lat'],item['lon'],item['region_id'],region])  

    df = pd.DataFrame(tmp)
    df.columns = ['station_id','name','latitude','longitude','region_idx','region']
    return df

In [125]:
df = get_bus_location()
df.to_csv('find_nearest_bus_station.csv')
df.head()

,station_id,name,latitude,longitude,region_idx,region
0,26,20th & E St NW,38.896300,-77.045000,42,"Washington, DC"
1,27,21st & I St NW,38.900800,-77.047000,42,"Washington, DC"
2,72,Georgetown Harbor / 30th St NW,38.902221,-77.059219,42,"Washington, DC"
3,91,13th St & New York Ave NW,38.900283,-77.029822,42,"Washington, DC"
4,106,17th & K St NW / Farragut Square,38.902061,-77.038322,42,"Washington, DC"


In [126]:
def find_neareast_bike_station(depart,dest):
    '''
    Args:   depart --> (latitude,longitude)
            dest --> (latitude,longitude)
            
    Return: depart_bike_station --> (latitude,longitude)
            dest_bike_station --> (latitude,longitude)
            info_depart --> (station_id of the bike station, name of the bike station)
            info_dest --> (station_id of the destination bike station, name of the destination bike station)
    '''
    min_depart_dist = 10**9
    coordinate_depart = []
    info_depart = {}
    # departure
    for i in range(df.shape[0]):
        x = df.loc[i,:]['latitude']
        y = df.loc[i,:]['longitude']
        depart_dist = ((x-depart[0])**2 + (y-depart[1])**2)**0.5
        if depart_dist < min_depart_dist:
            info_depart['station_id'] = int(df.loc[i,:]['station_id'])
            info_depart['name'] = df.loc[i,:]['name']
            
            min_depart_dist = depart_dist
            if len(coordinate_depart) == 1:
                coordinate_depart.pop()
            if len(coordinate_depart) == 0:
                coordinate_depart.append([depart[0],depart[1]])
                
            
            
    # destination
    min_dest_dist = 10**9
    coordinate_dest = []
    info_dest = {}
    for i in range(df.shape[0]):
        x = df.loc[i,:]['latitude']
        y = df.loc[i,:]['longitude']
        dest_dist = ((x-dest[0])**2 + (y-dest[1])**2)**0.5
        if dest_dist < min_dest_dist: 
            info_dest['station_id'] = int(df.loc[i,:]['station_id'])
            info_dest['name'] = df.loc[i,:]['name']
            
            min_dest_dist = dest_dist
            if len(coordinate_dest) == 1:
                coordinate_dest.pop()
            if len(coordinate_dest) == 0:
                coordinate_dest.append([dest[0],dest[1]])
                
    return coordinate_depart, coordinate_dest, info_depart, info_dest 

In [137]:
#test
%%time
depart,dest = [38.885300,-77.050000], [38.857766,-77.059580]
find_neareast_bike_station(depart,dest)

CPU times: user 404 ms, sys: 4.31 ms, total: 408 ms
Wall time: 407 ms
